# COMP30027 Machine Learning Assignment 2

## Description of text features

This notebook describes the pre-computed text features provided for assignment 2. **You do not need to recompute the features yourself for this assignment** -- this information is just for your reference. However, feel free to experiment with different text features if you are interested. If you do want to try generating your own text features, some things to keep in mind:
- There are many different decisions you can make throughout the feature design process, from the text preprocessing to the size of the output vectors. There's no guarantee that the defaults we chose will produce the best possible text features for this classification task, so feel free to experiment with different settings.
- These features must be trained on a training corpus. Generally, the training corpus should not include validation samples, but for the purposes of this assignment we have used the entire non-test set (training+validation) as the training corpus, to allow you to experiment with different validation sets. If you recompute the text features as part of your own model, you should exclude validation samples and retrain on training samples only. Note that if you do N-fold cross-validation, this means generating N sets of features for N different training-validation splits.
- This code may take a long time to run and require a good bit of memory, which is why we are not requiring you to recompute these features yourself. doc2vec in particular is very slow unless you can implement some speed-ups in C.

In [2]:
import numpy as np
import pandas as pd

# read text
# for DEMONSTRATION PURPOSES, the entire training set will be used to train the models and also as a test set
x_train_original = pd.read_csv(r"recipe_train.csv", index_col = False, delimiter = ',', header=0)
# use recipe name as an example
train_corpus_name = x_train_original['name']
test_name = x_train_original['name']

## Count vectorizer

A count vectorizer converts documents to vectors which represent word counts. Each column in the output represents a different word and the values indicate the number of times that word appeared in the document. The overall size of a count vector matrix can be quite large (the number of columns is the total number of different words used across all documents in a corpus), but most entries in the matrix are zero (each document contains only a few of all the possible words). Therefore, it is most efficient to represent the count vectors as a sparse matrix.

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

# preprocess text and compute counts
vocab_name = CountVectorizer(stop_words='english').fit(train_corpus_name)

# generate counts for a new set of documents
x_train_name = vocab_name.transform(train_corpus_name)
x_test_name = vocab_name.transform(test_name)

# check the number of words in vocabulary
print(len(vocab_name.vocabulary_))
# check the shape of sparse matrix
print(x_train_name.shape)

10892
(40000, 10892)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
# Test to see approximate accuracy by splitting the training data to train and test data = 7:3

data = pd.read_csv("recipe_train.csv")
X = data.iloc[:,:-1]
y = data.iloc[:,-1]
X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(X,y,test_size=0.33)

# train_n_steps = pd.DataFrame(X_train_raw.n_steps)
# train_n_steps.reset_index(drop=True, inplace=True)
# test_n_steps = pd.DataFrame(X_test_raw.n_steps)
# test_n_steps.reset_index(drop=True, inplace=True)

# train_n_ingredients = pd.DataFrame(X_train_raw.n_ingredients)
# train_n_ingredients.reset_index(drop=True, inplace=True)
# test_n_ingredients = pd.DataFrame(X_test_raw.n_ingredients)
# test_n_ingredients.reset_index(drop=True, inplace=True)

# print(X_train_raw.n_steps)
# count vectorizer on feature 'name'
print("Feature Name")
vec = CountVectorizer(stop_words='english')
print("Fit and transform")
X_train_name = vec.fit_transform(X_train_raw.name)
X_test_name = vec.transform(X_test_raw.name)
# put to dataframe
df_train_name = pd.DataFrame(X_train_name.todense(),columns = vec.get_feature_names())
df_test_name = pd.DataFrame(X_test_name.todense(),columns = vec.get_feature_names())
# print(X_train_name.shape, X_test_name.shape)
# print(len(X_train_raw.n_steps))

# count vectorizer on feature 'steps'
X_train_steps = vec.fit_transform(X_train_raw.steps)
X_test_steps = vec.transform(X_test_raw.steps)
df_train_steps = pd.DataFrame(X_train_steps.todense(),columns = vec.get_feature_names())
df_test_steps = pd.DataFrame(X_test_steps.todense(),columns = vec.get_feature_names())
# print(df_train_steps)

# count vectorizer on feature 'ingredients'
# X_train_ing = vec.fit_transform(X_train_raw.ingredients)
# X_test_ing = vec.transform(X_test_raw.ingredients)
# df_train_ing = pd.DataFrame(X_train_ing.todense(),columns = vec.get_feature_names())
# df_test_ing = pd.DataFrame(X_test_ing.todense(),columns = vec.get_feature_names())
# print(X_train_ing.shape, X_test_ing.shape)
print("Merging")
# put all into one dataframe
df_train_name_steps =  pd.merge(df_train_name, df_train_steps, how="outer")
print(df_train_name_steps)
# df_train = pd.merge(df_train_name.head(), df_train_steps.head(), how="outer")
# train = pd.concat([df_train_name, df_train_steps,df_train_ing,train_n_steps,train_n_ingredients],axis=1)
# test = pd.concat([df_test_name, df_test_steps,df_test_ing,test_n_steps,test_n_ingredients],axis=1)
# print(X_train_raw.n_steps)

# print(train)
# print(X_train_raw.n_steps.shape)

Feature Name
Fit and transform
Merging


In [4]:
from sklearn import svm
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import time
from time import ctime
models = [GaussianNB(),
          MultinomialNB(),
          svm.LinearSVC(),
          DecisionTreeClassifier(),
          LogisticRegression()]
titles = ['GNB',
          'MNB',
          'LinearSVC',
          'Decision Tree',
          'Logistic Regression']

for title, model in zip(titles, models):
    print(title)
    start = time.time()
    print("START ",start)
    model.fit(train,y_train_raw)
    acc = model.score(test,y_test_raw)
    end = time.time()
    print("END ",end)
    t = end - start
    print(title, "Accuracy:",acc, 'Time:', t," s")


GNB
START  1621171833.176519
END  1621172001.737783
GNB Accuracy: 0.44196969696969696 Time: 168.56126403808594  s
MNB
START  1621172001.7691991
END  1621172036.119162
MNB Accuracy: 0.7365909090909091 Time: 34.34996294975281  s
LinearSVC
START  1621172036.1207008


/Users/patriciaangelica/.pyenv/versions/3.7.3/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


END  1621172104.9481778
LinearSVC Accuracy: 0.7509090909090909 Time: 68.827476978302  s
Decision Tree
START  1621172104.949993
END  1621172165.90699
Decision Tree Accuracy: 0.7457575757575757 Time: 60.95699715614319  s
Logistic Regression
START  1621172165.908127


/Users/patriciaangelica/.pyenv/versions/3.7.3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


END  1621172880.784724
Logistic Regression Accuracy: 0.7946969696969697 Time: 714.8765969276428  s


## doc2vec

doc2vec methods are an extension of word2vec. word2vec maps words to a high-dimensional vector space in such a way that words which appear in similar contexts will be close together in the space. doc2vec does a similar embedding for multi-word passages. The doc2vec (or Paragraph Vector) method was introduced by:

**Le & Mikolov (2014)** Distributed Representations of Sentences and Documents<br>
https://arxiv.org/pdf/1405.4053v2.pdf

The implementation of doc2vec used for this project is from gensim and documented here:<br>
https://radimrehurek.com/gensim/models/doc2vec.html

The size of the output vector is a free parameter. Most implemementations use around 100-300 dimensions, but the best size depends on the problem you're trying to solve with the embeddings and the number of training samples, so you may wish to try different vector sizes. We provided three sets of doc2vec features, and their dimensions are 50 and 100. The vectors themselves represent directions in a high-dimensional concept space; the columns do not represent specific words or phrases. Values in the vector are continuous real numbers and can be negative.

In [6]:
import gensim

# size of the output vector
vec_size = 20

# function to preprocess and tokenize text
def tokenize_corpus(txt, tokens_only=False):
    for i, line in enumerate(txt):
        tokens = gensim.utils.simple_preprocess(line)
        if tokens_only:
            yield tokens
        else:
            yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

# tokenize a training corpus
corpus_name = list(tokenize_corpus(train_corpus_name))

# train doc2vec on the training corpus
model = gensim.models.doc2vec.Doc2Vec(vector_size=vec_size, min_count=2, epochs=40)
model.build_vocab(corpus_name)
model.train(corpus_name, total_examples=model.corpus_count, epochs=model.epochs)

# tokenize new documents
doc = list(tokenize_corpus(test_name, tokens_only=True))

# generate embeddings for the new documents
x_test_name = np.zeros((len(doc),vec_size))
for i in range(len(doc)):
    x_test_name[i,:] = model.infer_vector(doc[i])
    
# check the shape of doc_emb
print(x_test_name.shape)

(40000, 20)
